![image](bt_logo.png)



In this note, you will learn: 1. `bt.indicators`  2. `notify_trade`

完整的Backtrader框架实现的移动平均线交叉策略示例

### Let us try a simple moving average (SMA) strategy

- The Simple Moving Average is a technical indicator that calculates the average price of an asset over a specific period. It smooths out price fluctuations and helps traders identify trends more easily. The SMA is calculated by summing up the prices over the chosen time frame and dividing it by the number of periods.

- SMA Crossovers:

    - The SMA trading strategy often involves using two different moving averages, typically a short-term SMA and a long-term SMA. 
        - Commonly used combinations are the 50-day SMA and the 200-day SMA. 
        - The strategy revolves around observing the crossovers between these two moving averages.
    > 常用的组合是 50 日SMA和 200 日SMA。
    > 该策略围绕观察这两条移动平均线之间的交叉点展开。

    - Bullish Crossover (看涨交叉): 
        - When the short-term SMA (e.g., 50-day) crosses above the long-term SMA (e.g., 200-day), it's considered a bullish signal. 
        > 当短期SMA（如50日）上穿长期SMA（如200日）时，被视为看涨信号。

        - This crossover suggests that the short-term trend is gaining strength and could indicate a potential upward movement in the asset's price. 
        - Traders may see this as a buying opportunity.

    - Bearish Crossover (看跌交叉): 
        - On the other hand, when the short-term SMA crosses below the long-term SMA, it's considered a bearish signal. 
        > 另一方面，当短期SMA下穿长期SMA时，被视为看跌信号。
        
        - This crossover indicates that the short-term trend is weakening and could imply a potential downward movement in the asset's price. 
        - Traders may view this as a signal to sell or avoid buying the asset.

<img src="sma.png" alt="app-screen" width="800" />


In [1]:
import matplotlib.pyplot as plt 
import backtrader as bt
from datetime import datetime
import pandas as pd
# import pf_api

In [2]:
# Load data from CSV and filter for 2025-01-01 onwards
df = pd.read_csv('./MSFT.csv', index_col='Date', parse_dates=True)
df = df[df.index >= '2025-01-01']

* In Chapter 2.1, we introduce the backtest of the SMA crossover strategy.

In [ ]:
# Parameters
short_window = 10
long_window = 20
initial_cash = 100000

# Initialize strategy variables 手动计算移动平均线和交易信号
df['Short_MA'] = df['Close'].rolling(window=short_window).mean()
df['Long_MA'] = df['Close'].rolling(window=long_window).mean()
df['Position'] = 0
cash = initial_cash

# Implement the strategy
for i in range(long_window, len(df)):
    current_date = df.index[i]
    short_ma_curr = df['Short_MA'].iloc[i]
    long_ma_curr = df['Long_MA'].iloc[i]
    short_ma_prev = df['Short_MA'].iloc[i-1]
    long_ma_prev = df['Long_MA'].iloc[i-1]
    
    # Detect actual crossover: Short MA crosses above Long MA
    if (short_ma_curr > long_ma_curr and 
        short_ma_prev <= long_ma_prev and 
        df['Position'].iloc[i - 1] == 0):
        # Buy signal: Short MA crosses above Long MA and no position is open
        shares_to_buy = 1
        df.loc[df.index[i], 'Position'] = shares_to_buy
        exe_price = df['Close'].iloc[i]
        cash -= shares_to_buy * exe_price
        print (f'buy at date {current_date} with price {exe_price}')
        
    # Detect actual crossover: Short MA crosses below Long MA
    elif (short_ma_curr < long_ma_curr and 
          short_ma_prev >= long_ma_prev and 
          df['Position'].iloc[i - 1] > 0):
        # Sell signal: Short MA crosses below Long MA and a position is open
        cash += df['Position'].iloc[i - 1] * df['Close'].iloc[i]
        df.loc[df.index[i], 'Position'] = 0
        exe_price = df['Close'].iloc[i]
        print (f'sell at date {current_date} with price {exe_price}')
    else:
        df.loc[df.index[i], 'Position'] = df['Position'].iloc[i-1]
        
# Calculate portfolio value
Portfolio_Value = cash + (df.loc[df.index[-1],'Close'] * df.loc[df.index[-1],'Position'])

# Print the DataFrame with positions and portfolio value
print(f'Portfolio Value: {Portfolio_Value}')

buy at date 2025-03-27 00:00:00 with price 390.5799865722656
sell at date 2025-03-31 00:00:00 with price 375.3900146484375
buy at date 2025-04-02 00:00:00 with price 382.1400146484375
sell at date 2025-04-04 00:00:00 with price 359.8399963378906
buy at date 2025-04-23 00:00:00 with price 374.3900146484375
sell at date 2025-08-20 00:00:00 with price 505.7200012207031
buy at date 2025-09-19 00:00:00 with price 517.9299926757812
Portfolio Value: 100089.33999633789


* After learning the backtrader, we can use backtrader to do the same backtest for us.

In [4]:
pandasdata = bt.feeds.PandasData(
    dataname=df,
    open=0,
    high=1,
    low=2,
    close=3,
    volume=5
)

In [ ]:
import backtrader as bt
from datetime import datetime

# Backtrader基础版本的SMA交叉策略
class SMACrossoverStrategy(bt.Strategy):
    # if your data is long enough, you may choose short_period=50 and long_period=200.
    params = (
        ('short_period', 10),
        ('long_period', 20),
    )   # 定义策略参数（短期和长期周期）
    
    def log(self, txt, dt=None):    # 统一的日志记录功能
        ''' 
        Logging function for this strategy
        '''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self): # 初始化指标和状态变量
        self.data_close = self.data.close # this is the same as self.datas[0].close
        self.short_sma = 0
        self.long_sma = 0
        self.crossover = False
        self.short_sma_prev = None  # Initialize to None
        self.long_sma_prev = None   # Initialize to None
        
    def notify_order(self, order):  # 订单状态通知处理
        if order.status in [order.Completed]:
            # Use the execution date from the order, not the current bar date
            exec_date = bt.num2date(order.executed.dt).date()
            if order.isbuy():
                # Buy order was filled
                filled_price = order.executed.price
                print(f"{exec_date}, Buy order filled at price: {filled_price}")
            elif order.issell():
                # Sell order was filled
                filled_price = order.executed.price
                print(f"{exec_date}, Sell order filled at price: {filled_price}")

    def next(self): # 策略核心逻辑，在每个bar上执行
        current_date = self.datas[0].datetime.date(0)
        
        # len(self) denotes how many units(e.g. days) have passed until now
        if len(self) < self.params.long_period: # if the rolling window is not full, pass
            return

        # Calculate Simple Moving Averages
        self.short_sma = sum(self.data.close.get(size=self.params.short_period)) / self.params.short_period
        self.long_sma = sum(self.data.close.get(size=self.params.long_period)) / self.params.long_period
        
        # Check for crossover: detect actual crossover from below to above
        # Only check crossover if we have previous values
        if (not self.crossover and 
            self.short_sma > self.long_sma and 
            self.short_sma_prev is not None and
            self.short_sma_prev <= self.long_sma_prev):
            self.buy(size=100)
            self.crossover = True
            
        elif (self.crossover and 
              self.short_sma < self.long_sma and 
              self.short_sma_prev is not None and
              self.short_sma_prev >= self.long_sma_prev):
            self.sell(size=100)
            self.crossover = False
        
        # Store current values as previous for next iteration
        self.short_sma_prev = self.short_sma
        self.long_sma_prev = self.long_sma

if __name__ == '__main__':
    cerebro = bt.Cerebro()  # 创建Cerebro引擎
    cerebro.broker.setcash(100000.0)    # 设置初始资金
    cerebro.adddata(pandasdata, name='MSFT')    # 添加数据
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.addstrategy(SMACrossoverStrategy)   # 添加策略
    cerebro.run()   # 运行回测

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Final Cash Value: %.2f' % cerebro.broker.getcash())

Starting Portfolio Value: 100000.00
2025-03-28, Buy order filled at price: 388.0799865722656
2025-04-01, Sell order filled at price: 374.6499938964844
2025-04-03, Buy order filled at price: 374.7900085449219
2025-04-07, Sell order filled at price: 350.8800048828125
2025-04-24, Buy order filled at price: 375.7000122070313
2025-08-21, Sell order filled at price: 503.6900024414063
2025-09-22, Buy order filled at price: 515.5900268554688
Final Portfolio Value: 108849.00
Final Cash Value: 57506.00


### bt.indicators

- [bt.indicators](https://www.backtrader.com/docu/induse/) provides a collection of pre-built technical indicators that traders can use to analyze financial data and develop trading strategies. These indicators are widely used in quantitative finance and technical analysis to identify patterns, trends, and potential trading signals in historical price data.

- Backtrader offers a wide range of indicators that can be easily integrated into your trading strategies. These indicators cover various aspects of market analysis, including trend identification, momentum, volatility, volume analysis, and more. Here are some of the commonly used indicators provided by bt.indicators,
    - Moving Averages:

        - Simple Moving Average (SMA, 简单移动平均线)
        - Exponential Moving Average (EMA, 指数移动平均线)
        - Weighted Moving Average (WMA, 加权移动平均线)
        - Smoothed Moving Average (SMMA, 平滑移动平均线)
        
    - Bollinger Bands: A volatility indicator that consists of a middle SMA line and two standard deviation bands above and below it.
    > 布林带：一种波动性指标，由一条中间的SMA线和上下两条标准差带组成。

    - Relative Strength Index (RSI): A momentum oscillator that measures the speed and change of price movements.
    > 相对强弱指数：一种动量振荡器，衡量价格变动的速度和变化。

    - Moving Average Convergence Divergence (MACD): A trend-following momentum indicator that shows the relationship between two EMAs.
    > 移动平均收敛散度：一种趋势跟踪动量指标，显示两条EMA之间的关系。
    
- You may refer to the official document [here](https://www.backtrader.com/docu/induse/) for more indicators and details.


- We can rewrite the previous SMA trading strategy as below:

In [ ]:
# Backtrader优化版本（使用bt.indicators）
class SMACrossoverStrategy(bt.Strategy):
    params = (
        ('short_period', 10),
        ('long_period', 20),
    )
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.short_sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.short_period)
        self.long_sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.long_period)
        self.crossover = bt.indicators.CrossOver(self.short_sma, self.long_sma)
        self.cross_flag = False

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            # 买单/卖单已提交/被经纪商接受 - 无需操作
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        # 注意：如果现金不足，经纪商可能拒绝订单
        if order.status in [order.Completed]:
            # Use actual execution date from order
            exec_date = bt.num2date(order.executed.dt).date()
            if order.isbuy():
                print(f'{exec_date}, BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                print(f'{exec_date}, Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def next(self):
        if not self.cross_flag and self.crossover > 0:  # Short-term SMA crosses above long-term SMA
            self.buy(size=100)
            self.cross_flag = True

        elif self.cross_flag and self.crossover < 0:  # Short-term SMA crosses below long-term SMA
            self.sell(size=100)
            self.cross_flag = False

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100000.0)
    cerebro.adddata(pandasdata, name='MSFT')
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.addstrategy(SMACrossoverStrategy)
    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2025-03-28, BUY EXECUTED, 388.0799865722656, 100 shares
2025-04-01, Sell EXECUTED, 374.6499938964844, -100 shares
2025-04-03, BUY EXECUTED, 374.7900085449219, 100 shares
2025-04-07, Sell EXECUTED, 350.8800048828125, -100 shares
2025-04-24, BUY EXECUTED, 375.7000122070313, 100 shares
2025-08-21, Sell EXECUTED, 503.6900024414063, -100 shares
2025-09-22, BUY EXECUTED, 515.5900268554688, 100 shares
Final Portfolio Value: 108849.00


#### bt.indicators 模块

这是Backtrader的技术指标库，提供：

- 移动平均线：SMA, EMA, WMA等
- 振荡指标：RSI, MACD, Stochastic等
- 趋势指标：Bollinger Bands, ADX等
- 便利工具：CrossOver（检测两条线交叉）

---

### Notify Trade

* In Backtrader, the Trade object represents a single trade executed during the backtesting process. It provides information about the entry and exit points of the trade, the size of the position, the price at which the trade was executed, and other relevant details.

* Detailed introduction about the `trade` object can be found [here](https://www.backtrader.com/docu/trade/)


* In Backtrader, the `notify_trade` method is part of a strategy's callback system and is called every time a trade is completed. A trade in Backtrader represents a single buy and sell sequence for a particular asset or instrument.

* The notify_trade method is useful for tracking and analyzing individual trades within your strategy. It allows you to access information about the completed trade, such as the profit or loss, duration, and other trade-specific metrics. You can use this method to perform various actions or record trade-related information.

In [ ]:
class SMACrossoverStrategy(bt.Strategy):
    params = (
        ('short_period', 10),
        ('long_period', 20),
    )
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.short_sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.short_period)
        self.long_sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.long_period)
        self.crossover = bt.indicators.CrossOver(self.short_sma, self.long_sma)
        self.cross_flag = False

    def notify_order(self, order):
        """
        订单状态回调函数，处理订单生命周期：
        """
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status == order.Completed:
            # Use actual execution date from order
            exec_date = bt.num2date(order.executed.dt).date()
            if order.isbuy():
                print(f'{exec_date}, BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                print(f'{exec_date}, Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        """
        交易完成回调函数，提供交易分析：
        """
        if not trade.isclosed:
            return
        
        # pnl (float): current profit and loss of the trade (gross pnl)
        # pnlcomm (float): current profit and loss of the trade minus commission (net pnl)  
        # barlen (int): number of bars this trade was open
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl}, NET {trade.pnlcomm}, Bars Passed {trade.barlen}')
        
    def next(self):
        # Short-term SMA crosses above long-term SMA
        if not self.cross_flag and self.crossover > 0:  
            self.buy(size=100)
            self.cross_flag = True
            
        # Short-term SMA crosses below long-term SMA
        elif self.cross_flag and self.crossover < 0:  
            self.sell(size=100)
            self.cross_flag = False

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(100000.0)
    # Add your desired slippage 
    cerebro.broker.set_slippage_perc(0.01)  # 1% slippage
    cerebro.broker.setcommission(commission=0.001)  # Modify commission if needed

    # Add your data source (data) here
    cerebro.adddata(pandasdata, name='MSFT')

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.addstrategy(SMACrossoverStrategy)
    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2025-03-28, BUY EXECUTED, 389.1300048828125, 100 shares
2025-04-01, Sell EXECUTED, 373.2300109863281, -100 shares
2025-04-01, OPERATION PROFIT, GROSS -1589.9993896484375, NET -1666.2353912353515, Bars Passed 2
2025-04-03, BUY EXECUTED, 377.4800109863281, 100 shares
2025-04-07, Sell EXECUTED, 347.3712048339844, -100 shares
2025-04-07, OPERATION PROFIT, GROSS -3010.880615234373, NET -3083.3657368164045, Bars Passed 2
2025-04-24, BUY EXECUTED, 379.45701232910164, 100 shares
2025-08-21, Sell EXECUTED, 502.7200012207031, -100 shares
2025-08-21, OPERATION PROFIT, GROSS 12326.298889160149, NET 12238.081187805168, Bars Passed 82
2025-09-22, BUY EXECUTED, 517.739990234375, 100 shares
Final Portfolio Value: 107005.71


For the first trade, the loss is below:

In [8]:
print (f'Gross Profit: {373.23*100-389.13*100}')
print (f'Net Profit: {373.23*100-389.13*100-373.23*100*.001-389.13*100*.001}')

Gross Profit: -1590.0
Net Profit: -1666.236
